In [1]:
%%time

from premise import *
import bw2data
bw2data.projects.set_current("ecoinvent-3.10.1-cutoff")
#bw2data.projects.set_current("ecoinvent-3.9.1-cutoff_bw25")
# clear_inventory_cache()
scenarios = [
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2020},
    {"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2040},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.10.1-cutoff",
        source_version="3.10",
        biosphere_name="biosphere3",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)
#ndb.write_db_to_brightway("test final energy 2")

Loading variables from /Users/romain/Github/premise/dev/variables.yaml
USER_DATA_BASE_DIR: test
premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                             

In [2]:
%%time
ndb.update("cement")

Processing scenarios for sector 'cement': 100%|█| 1/1 [00:08<00:00,  8

Done!

CPU times: user 7.72 s, sys: 307 ms, total: 8.03 s
Wall time: 8.12 s


In [3]:
ndb.write_db_to_brightway("test cement 3")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
---> MAJOR anomalies found: check the change report.


KeyError: "Can't find linking activity for exchange:\n{'amount': 0.73341,\n 'location': 'CAZ',\n 'name': 'carbon dioxide, captured, at cement production plant, using '\n         'monoethanolamine',\n 'product': 'carbon dioxide, captured',\n 'type': 'technosphere',\n 'unit': 'kilogram'}"

In [9]:
import bw2calc
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
db = bw2data.Database("test steel 9")
act = db.random()
method = bw2data.methods.random()
print(act)
print(method)
lca = bw2calc.LCA({act:1}, method)
lca.lci()
lca.lcia()
lca.score

'pig iron production, smelting reduction' (kilogram, USA, None)
('IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')


0.0

In [16]:
lca.inventory

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 335965 stored elements and shape (2687, 26329)>

In [3]:
from premise.utils import load_database

for scenario in ndb.scenarios:
    scenario = load_database(scenario)

In [4]:
from premise.activity_maps import InventorySet, debug_mapping_to_dataframe

In [5]:
import pandas as pd
# Display full DataFrame without truncation
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.max_colwidth', None)    # Don't truncate column content
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping


In [5]:
inv = InventorySet(
    database=ndb.scenarios[0]["database"],
    version="3.10",
    model="remind"
)
biomass_map = inv.generate_cement_map()
df_biomass = debug_mapping_to_dataframe(biomass_map)
df_biomass


,Category,Market,Product,Locations
0,"cement, dry feed rotary kiln",clinker production,clinker,"[BR, CA-QC, CAZ, CH, CHA, CO, EC, EUR, Europe ..."
1,"cement, dry feed rotary kiln, efficient, with ...",clinker production,clinker,"[BR, CA-QC, CAZ, CH, CHA, CO, EC, EUR, Europe ..."
2,,"clinker production, dry feed rotary kiln, effi...",clinker,"[CAZ, CHA, EUR, Europe without Switzerland, IN..."


In [ ]:
inv = InventorySet(database=my_database)
biomass_map = inv.generate_biomass_map()


In [9]:
biomass = {k: list(set([(x["name"], x["reference product"]) for x in v])) for k, v in inv.generate_biomass_map().items()}
#generate_heat_map
#generate_activities_using_metals_map
#generate_powerplant_map
#generate_cdr_map
#generate_powerplant_fuels_map
#generate_steel_map
#generate_fuel_map
#generate_mining_waste_map
#generate_final_energy_map


In [6]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if not isinstance(e["amount"], (float, int)):
            print(e)

In [7]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].startswith("market for clinker") and ds["location"] == "EUR":
        for e in ds["exchanges"]:
            print(e["name"], e["type"], e.get("location"), e["amount"])
        print()
        #from pprint import pprint
        #pprint(ds)
        #break

market for clinker production EUR 1.0
clinker production technosphere EUR 0.8066325749282948
clinker production, dry feed rotary kiln, efficient, with MEA CCS technosphere EUR 0.19336742507170518



In [ ]:
FU = [
    {a: 1} for a in
    bw2data.Database("ecoinvent_cutoff_3.9_image_SSP2-Base_2050")
    if a["name"].startswith("electricity production") and a["unit"] == "kilowatt hour"
]

print(len(FU))

list_meth = [
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H'),
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')
]
import bw2calc
bw2data.calculation_setups['multiLCA'] = {'inv': FU, 'ia': list_meth}
myMultiLCA = bw2calc.MultiLCA('multiLCA')
df_impact_exc = pd.DataFrame(columns = [", ".join(i) for i in list_meth],
     data = myMultiLCA.results,
     index=[k["name"] + ", " + k["location"] for i in FU for k in i]
)

In [ ]:
ndb.update_steel()

In [ ]:
ndb.update_biomass()

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.write_db_to_simapro()

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].startswith("market group for electricity, low"):
        print(ds["name"], ds["reference product"], ds["location"])

In [ ]:
ndb.scenarios[1]["iam data"].carbon_capture_rate

In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["reference product"]=="biogas":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"))
        print()



In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[1]["database"]:
    if ds["name"]=="market group for heat, district or industrial, natural gas":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"), e["type"])
        print()


In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["name"]=="transport, freight, lorry, plugin diesel hybrid, 26t gross weight, 2020, EURO-VI, regional delivery" and ds["location"] == "ME":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"), e["type"])
        print()


In [ ]:
from premise.geomap import Geomap
geo = Geomap("remind")
geo.ecoinvent_to_iam_location("Europe, without Russia and Turkey")

In [ ]:
geo.iam_to_ecoinvent_location("EUR")

In [ ]:
geo.rev_additional_mappings

In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[0]["database"]:
    if ds["name"]=="pig iron production" and ds["location"] == "CHN":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])
        print(ds["comment"])
        print()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame.from_dict(res, orient="index").plot(kind="bar", figsize=(20, 5))
plt.ylabel("kg heavy fuel oil/kWh")
plt.savefig("electricity production, oil.png")

In [ ]:
ndb.scenarios[0]["iam data"].electricity_efficiencies.coords["variables"].values

In [ ]:
ndb.scenarios[0]["iam data"].electricity_efficiencies.sel(variables="Biomass IGCC CCS", year=2050)

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.scenarios[0]["iam data"].gas_markets.sel(region="World")

In [ ]:
ndb.scenarios[0].

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"] == "market for platinum" and e["product"] == "platinum":
            print(ds["name"], ds["location"], e["name"], e["location"])

In [ ]:
import pprint
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].lower() == "lithium carbonate, precipitated":
        print(ds["name"], ds["location"])

In [ ]:
from premise.activity_maps import InventorySet
inv = InventorySet(ndb.database, "3.9")

In [ ]:
inv.generate_metals_map()

In [ ]:
inv.generate_powerplant_map()

In [ ]:
inv.generate_daccs_map()

In [ ]:
ndb.write_db_to_brightway("test_metals")

In [ ]:
products = [
    'lightweighting, glider',
    'auxilliary battery inputs, for road vehicle',
    'battery cell, Li-ion, NMC622',
    'auxilliary electronics inputs, for fuel cell electric vehicle',
    'auxilliary material inputs, for fuel cell electric vehicle',
    'fuel cell stack, for fuel cell electric vehicle',
    'carbon fiber, weaved',
    'tank liner, polyethylene, high density',
    'transformer and rectifier unit, for electrolyser',
    'fuel tank, for diesel road vehicle',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed natural gas, 200 bar',
    'charger, level 3, plugin, 200 kW, for electric vehicle',
    'gearbox, for lorry',
    'transmission, for lorry',
    'suspension, for lorry',
    'power electronics, for lorry',
    'auxilliary material inputs, for electric lorry',
    'retarder, for lorry',
    'cabin, for lorry',
    'frame, blanks and saddle, for lorry',
    'tires and wheels, for lorry',
    'auxilliary material inputs, for hydrid electric lorry',
    'exhaust system, for lorry',
    'internal combustion engine, for lorry',
    'lead acid battery, for lorry',
    'charger, level 3, with pantograph, 450 kW, for electric vehicle',
    'overhead lines, for electric vehicle',
    'battery cell, Li-ion, LTO',
    'charging station, 3kW',
    'battery cell,  Li-ion, LFP',
    'battery cell, Li-ion, NCA',
    'assembly operation, for lorry'
]

print(len(products))

acts = [a for a in bw.Database("brian") if a["reference product"] in products]

In [ ]:
len(acts)

In [ ]:
[a["reference product"] for a in acts]

In [ ]:
import bw2data
bw2io.BW2Package.export_obj(obj=bw.Database("brian"), filename="db_for_brian.bw2package")

In [ ]:
import bw2data
bw2io.BW2Package.import_file("db_for_brian.bw2package")

In [ ]:
fp = "/Users/romain/Library/Application Support/Brightway3/ei39.f1469ad5eb0a071dc60323f6e0fce980/output/lci-brian.xlsx"
import bw2io

i = bw2io.ExcelImporter(fp)
i.apply_strategies()

In [ ]:
ei_acts = list(set([(a["name"], a["reference product"]) for a in bw.Database("ecoinvent 3.9.1 cutoff")]))
print(len(ei_acts))

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) in ei_acts:
        i.data.remove(ds)

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) not in ei_acts:
        print(ds["name"])

In [ ]:
len(i.data)

In [ ]:
for ds in i.data:
    print(ds["name"])

In [ ]:
i.match_database(fields=["name", "unit", "location"])
i.match_database("ecoinvent 3.9.1 cutoff", fields=["name", "reference product", "location", "unit"])
i.match_database("biosphere3", fields=["name", "categories", "unit"])

In [ ]:
i.statistics()

In [ ]:
list(i.unlinked)[-1]

In [ ]:
bw.get_activity(('brian', '360c5e612a154ebb8cee2650bcc1bc75_copy1'))

In [ ]:
for f in bw.Database("biosphere3"):
    if f["code"] == "e030108f-2125-4bcb-a73b-ad72130fcca3":
        print(f)

In [ ]:
bw.projects.set_current("ei39")

In [ ]:
l = []

for f in bw.Database("biosphere3"):
    l.append([f["name"], f["categories"][0], f["categories"][1] if len(f["categories"])>1 else "unspecified", f["unit"], f["code"]])

In [ ]:
import csv

with open("flows_biosphere_39.csv", "w", newline="", ) as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(l)

In [ ]:
ndb.scenarios[1]["iam data"].fuel_markets.sel(region="NEU").variables.values

In [ ]:
import numpy as np
for ds in ndb.scenarios[1]["database"]:
    if ds["name"] == ("carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with waste heat, and grid electricity",
                            "carbon dioxide, captured from atmosphere"):
            print(ds["name"], ds["location"])

In [ ]:
for ds in ndb.scenarios[1]["database"]:
    if "carbon dioxide, captured" in ds["name"]:
        print(ds["name"], ds["location"])

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU", year=2050)

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU")

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.update_all()

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.scenarios[0]["iam data"].gains_data_IAM

In [ ]:
ndb.update_all()

In [ ]:
ndb.write_db_to_brightway(["test_dac", "test_dac2"])

In [ ]:
from premise.geomap import Geomap

In [ ]:
Geomap()

In [ ]:
Geomap(model="remind").geo.keys()

In [ ]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="steel")

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"].startswith("CO2 capture, at cement production plant"):
            print(e["name"], e.get("location"), e["amount"])
            print(ds["name"])
            print("--------------------")